In [ ]:
!ls

In [ ]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from functools import lru_cache
import time

# 🔽 Load both datasets
df_2024 = pd.read_csv("./filtered_BexarMay24_hotel_tax_receipts.csv")
df_2025 = pd.read_csv("./filtered_BexarMay25_hotel_tax_receipts.csv")

# 🔗 Merge on hotel name + address fields (adjust join keys if needed)
df = pd.merge(
    df_2024,
    df_2025,
    on=["Location_Name", "Location_Address", "Location_City", "Location_Zip", "Unit_Capacity"],
    suffixes=("_2024", "_2025")
)

# ➕ Calculate revenue difference
df["Revenue_Diff"] = df["Total_Room_Receipts_2025"] - df["Total_Room_Receipts_2024"]

# 🧭 Setup geocoder for local Nominatim
geolocator = Nominatim(user_agent="tx_hotels_map", domain="localhost:8080", scheme="http")

# 💾 Cache geocoding results by address
@lru_cache(maxsize=5000)
def cached_geocode(address):
    try:
        return geolocator.geocode(address, timeout=3)
    except:
        return None

# 📌 Apply geocoding
def get_coordinates(row):
    full_address = f"{row['Location_Address']}, {row['Location_City']}, TX {row['Location_Zip']}"
    location = cached_geocode(full_address)
    if location:
        return pd.Series([location.latitude, location.longitude])
    return pd.Series([None, None])

df[["Latitude", "Longitude"]] = df.apply(get_coordinates, axis=1)

# 🗺️ Create Folium Map
hotel_map = folium.Map(location=[29.5187, -98.6047], zoom_start=12)

# Scale marker size
min_receipt = df["Total_Room_Receipts_2025"].min()
max_receipt = df["Total_Room_Receipts_2025"].max()
range_receipt = max_receipt - min_receipt

for _, row in df.dropna(subset=["Latitude", "Longitude"]).iterrows():
    scaled_radius = 4 + 16 * ((row["Total_Room_Receipts_2025"] - min_receipt) / range_receipt)

    color = (
        "green" if row["Revenue_Diff"] > 0 else
        "red" if row["Revenue_Diff"] < 0 else
        "gray"
    )

    popup_text = f"""
    <b>{row['Location_Name']}</b><br>
    2024 Receipts: ${row['Total_Room_Receipts_2024']:,.0f}<br>
    2025 Receipts: ${row['Total_Room_Receipts_2025']:,.0f}<br>
    Difference: <b style='color:{color}'>${row['Revenue_Diff']:,.0f}</b><br>
    Units: {row['Unit_Capacity']}
    """

    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=scaled_radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(hotel_map)

# 💾 Save Map
hotel_map.save("Hotel_Revenue_2024_2025_Comparison.html")
